1.1 Import necessary Python packages

In [ ]:
# download brainiak package
! python3 -m pip install -U brainiak

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 7.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.0-py3-none-any.whl (227 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 72.7 MB/s eta 0:00:00
  Created wheel for brainiak: filename=brainiak-0.11-cp310-cp310-linux_x86_64.whl size=1138211 sha256=b518938e64755

In [ ]:
# download nilearn
!pip install -U nilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 104.1 MB/s eta 0:00:00
  Attempting uninstall: nibabel
    Found existing installation: nibabel 3.0.2
    Uninstalling nibabel-3.0.2:
      Successfully uninstalled nibabel-3.0.2


In [ ]:
# download nibabel
!pip install git+https://github.com/nipy/nibabel

  Cloning https://github.com/nipy/nibabel to /tmp/pip-req-build-8lwzmzr_
  Running command git clone --filter=blob:none --quiet https://github.com/nipy/nibabel /tmp/pip-req-build-8lwzmzr_
  Resolved https://github.com/nipy/nibabel to commit 6fe3e6b17857a5bba9e8fb31684d606dd458a6f1
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nibabel: filename=nibabel-5.2.0.dev20+g6fe3e6b1-py3-none-any.whl size=3289401 sha256=b7ca233bb952c243fea4caf1b72934d4e52b9a6bcb0e87209dfa382a8f31a1b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-kebrlc5z/wheels/5d/78/3b/248c139b24ead344e02b5b5d37a1b631e5e97be0770b7a153d
Successfully built nibabel
  Attempting uninstall: nibabel
    Found existing installation: nibabel 5.1.0
    Uninstalling nibabel-5.1.0:
      Successfully uninstalled nibabel-5.1.0


In [ ]:
# import packages
from pathlib import Path
from brainiak.utils import fmrisim
import nibabel
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage
import scipy.spatial.distance as sp_distance
import sklearn.manifold as manifold
import scipy.stats as stats
import sklearn.model_selection
import sklearn.svm

1.2 Load participant data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
home = '/content/drive/MyDrive'
nii = nibabel.load(home + '/Corr_MVPA/Participant_01_rest_run01.nii')
volume = nii.get_data()

<ipython-input-4-89a5975caeeb>:3: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  volume = nii.get_data()


In [ ]:
dim = volume.shape  # What is the size of the volume
dimsize = nii.header.get_zooms()  # Get voxel dimensions from the nifti header
tr = dimsize[3]
if tr > 100:  # If high then these values are likely in ms and so fix it
    tr /= 1000
print('Volume dimensions:', dim)
print('TR duration: %0.2fs' % tr)
print(dimsize)

Volume dimensions: (64, 64, 27, 294)
TR duration: 1.50s
(3.0, 3.0, 3.5000014, 1500.0)


1.3 Generate an activity template and a mask

In [ ]:
mask, template = fmrisim.mask_brain(volume=volume,
                                    mask_self=True,
                                    )

In [ ]:
print(mask.shape)
print(template.shape)

(64, 64, 27)
(64, 64, 27)


In [ ]:
# unzip probability maps of gray matter, white matter, and CSF respectively
!unzip drive/MyDrive/dataset/sub-01_T1w_class-GM_probtissue.nii.gz

Archive:  drive/MyDrive/dataset/sub-01_T1w_class-GM_probtissue.nii.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of drive/MyDrive/dataset/sub-01_T1w_class-GM_probtissue.nii.gz or
        drive/MyDrive/dataset/sub-01_T1w_class-GM_probtissue.nii.gz.zip, and cannot find drive/MyDrive/dataset/sub-01_T1w_class-GM_probtissue.nii.gz.ZIP, period.


In [ ]:
!unzip drive/MyDrive/dataset/sub-01_T1w_class-WM_probtissue.nii.gz

Archive:  drive/MyDrive/dataset/sub-01_T1w_class-WM_probtissue.nii.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of drive/MyDrive/dataset/sub-01_T1w_class-WM_probtissue.nii.gz or
        drive/MyDrive/dataset/sub-01_T1w_class-WM_probtissue.nii.gz.zip, and cannot find drive/MyDrive/dataset/sub-01_T1w_class-WM_probtissue.nii.gz.ZIP, period.


In [ ]:
!unzip drive/MyDrive/dataset/sub-01_T1w_class-CSF_probtissue.nii.gz

Archive:  drive/MyDrive/dataset/sub-01_T1w_class-CSF_probtissue.nii.gz
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of drive/MyDrive/dataset/sub-01_T1w_class-CSF_probtissue.nii.gz or
        drive/MyDrive/dataset/sub-01_T1w_class-CSF_probtissue.nii.gz.zip, and cannot find drive/MyDrive/dataset/sub-01_T1w_class-CSF_probtissue.nii.gz.ZIP, period.


In [ ]:
import nibabel as nib
# load functional data
filepath_func = 'drive/MyDrive/dataset/sub-01_ses-localizer_task-objectcategories_run-1_bold_space-T1w_preproc.nii.gz'
func = nib.load(filepath_func)

In [ ]:
# load gray matter
filepath_gm = 'drive/MyDrive/dataset/sub-01_T1w_class-GM_probtissue.nii.gz'
gm = nib.load(filepath_gm)
print(gm.shape)
print(gm.affine)

(274, 384, 384)
[[ 6.93359315e-01  4.50425558e-02  7.98166469e-02 -1.16284210e+02]
 [-3.23181227e-02  6.55866504e-01 -1.15482718e-01 -6.84598312e+01]
 [-9.06430706e-02  1.10701263e-01  6.51718795e-01 -1.61474640e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
#import nibabel.affines
from nibabel.affines import rescale_affine
import nibabel.processing as nibp
# resize gray matter
func0 = func.slicer[:,:,:,0]
print(func0)
print(func0.shape)
print(func0.affine)
gm_funcSize=nibp.resample_from_to(gm, func0, order=1)

ImportError: ignored

In [ ]:
gm.shape

(274, 384, 384)

In [ ]:
# discretize gray matter
gm_values = gm_funcSize.get_fdata()
gm_mask = (gm_values>0.1)

NameError: ignored

In [ ]:
print(np.count_nonzero(gm_mask))
print(gm_mask.size)

NameError: ignored

In [ ]:
print(gm_values.max())
print(gm_values.min())

In [ ]:
gm_mask_large = (gm_values>1)
gm_mask_small = (gm_values<0.1)

In [ ]:
plt.figure()
# plt.imshow(gm_mask_large[:, :, 26], cmap=plt.cm.gray)
plt.imshow(gm_mask_small[:, :, 26], cmap=plt.cm.gray, alpha=0.2)
# plt.imshow(gm_mask[:, :, 26], cmap=plt.cm.gray, alpha=0.2)
plt.axis('off')



In [ ]:
# load white matter and csf
filepath_wm = 'drive/MyDrive/dataset/sub-01_T1w_class-WM_probtissue.nii.gz'
wm = nib.load(filepath_wm)
print(wm.shape)
print(wm.affine)
filepath_csf = 'drive/MyDrive/dataset/sub-01_T1w_class-CSF_probtissue.nii.gz'
csf = nib.load(filepath_csf)
print(wm.shape)
print(csf.affine)

In [ ]:
# resize white matter and csf
wm_funcSize=nibp.resample_from_to(wm, func0, order=1)
csf_funcSize=nibp.resample_from_to(csf, func0, order=1)

In [ ]:
nib.save(wm_funcSize, os.path.join('drive/MyDrive/dataset', 'wm_funcSize_2.nii.gz'))
nib.save(csf_funcSize, os.path.join('drive/MyDrive/dataset', 'csf_funcSize_2.nii.gz'))

In [ ]:
# discretize white matter and csf and make confounds mask
wm_values = wm_funcSize.get_fdata()
csf_values = csf_funcSize.get_fdata()
confounds_values = wm_values+csf_values
confounds_mask = (confounds_values>0.1)

In [ ]:
print(wm_values.max())
print(wm_values.min())
print(csf_values.max())
print(csf_values.min())
print(confounds_values.max())
print(confounds_values.min())

In [ ]:
template_all = gm_values + wm_values + csf_values
print(template_all.max())
print(template_all.min())

In [ ]:
plt.figure()
# plt.imshow(template_all[:, :, 26], cmap=plt.cm.gray)
# plt.imshow(confounds_mask[:, :, 26], cmap=plt.cm.gray, alpha=0.5)
plt.imshow(gm_mask[:, :, 26], cmap=plt.cm.gray, alpha=0.2)
plt.axis('off')

In [ ]:
print(np.count_nonzero(confounds_mask))
print(confounds_mask.size)

In [ ]:
# brain_mask = gm_mask | confounds_mask
diff = gm_mask & confounds_mask
gm_mask_c = gm_mask ^ diff
confounds_mask_c = confounds_mask ^diff
brain_mask = gm_mask_c | confounds_mask_c
print(np.count_nonzero(gm_mask_c))
print(np.count_nonzero(confounds_mask_c))
print(np.count_nonzero(brain_mask))

In [ ]:
plt.figure()
# plt.imshow(template_all[:, :, 26], cmap=plt.cm.gray)
plt.imshow(confounds_mask_c[:, :, 26]*0.5, cmap='magma',vmin=0, vmax=1)
plt.imshow(gm_mask_c[:, :, 26], cmap='magma',alpha=0.5,vmin=0, vmax=1)
plt.axis('off')

1.4 Determine noise parameters

In [ ]:
# Calculate the noise parameters from the data. Set it up to be matched.
noise_dict = {'voxel_size': [dimsize[0],dimsize[1],dimsize[2]], 'matched': 1}
noise_dict = fmrisim.calc_noise(volume=volume,
                                mask=mask,
                                template=template,
                                noise_dict=noise_dict,
                                )

In [ ]:
print('Noise parameters of the data were estimated as follows:')
print('SNR: ' + str(noise_dict['snr']))
print('SFNR: ' + str(noise_dict['sfnr']))
print('FWHM: ' + str(noise_dict['fwhm']))

Noise parameters of the data were estimated as follows:


NameError: ignored

In [ ]:
# Create the different types of noise
total_time = 500
timepoints = list(range(0, total_time, int(tr)))

drift = fmrisim._generate_noise_temporal_drift(total_time,
                                               int(tr),
                                               )

mini_dim = np.array([2, 2, 2])
autoreg = fmrisim._generate_noise_temporal_autoregression(timepoints,
                                                          noise_dict,
                                                          mini_dim,
                                                          np.ones(mini_dim),
                                                          )

phys = fmrisim._generate_noise_temporal_phys(timepoints,
                                            )

stimfunc = np.zeros((int(total_time / tr), 1))
stimfunc[np.random.randint(0, int(total_time / tr), 50)] = 1
task = fmrisim._generate_noise_temporal_task(stimfunc,
                                            )

In [ ]:
# Plot the different noise types
plt.figure(figsize=(30,7))
plt.title('Noise types')

plt.subplot(2, 2, 1)
plt.plot(drift,color='k',linewidth=3)
plt.axis('off')
plt.xlabel('Drift')

plt.subplot(2, 2, 2)
plt.plot(autoreg[0, 1, 0, :],color='k',linewidth=1.5)
plt.axis('off')
plt.xlabel('Autoregression')

plt.subplot(2, 2, 3)
plt.plot(phys,color='k',linewidth=2)
plt.axis('off')
plt.xlabel('Physiological')

plt.subplot(2, 2, 4)
plt.plot(task,color='k',linewidth=2)
plt.axis('off')
plt.xlabel('Task')
plt.savefig('drive/MyDrive/dataset/fig3/noise.png',dpi=800)

**2. Generate noise**

In [ ]:
volume_c = (func.shape)
print(volume_c)
print(brain_mask.shape)
dimsize_c = func.header.get_zooms()
print(dimsize_c)

In [ ]:
brain_mask_c = 1*brain_mask

In [ ]:
print(volume_c[0:3])
print(dimsize_c[3])
print(dim[3])

In [ ]:
# Calculate the noise given the parameters
noise = fmrisim.generate_noise(dimensions=volume_c[0:3],
                               tr_duration=int(dimsize_c[3]),
                               stimfunction_tr=[0] * volume_c[3],
                               mask=brain_mask,
                               template=template_all,
                               noise_dict=noise_dict,
                               )

In [ ]:
print(noise.shape)
noise_reshaped = np.reshape(noise,[noise.shape[0]*noise.shape[1]*noise.shape[2],noise.shape[3]])
volume_reshaped = np.reshape(volume,[volume.shape[0]*volume.shape[1]*volume.shape[2],volume.shape[3]])
print(noise_reshaped.shape)
print(volume_reshaped.shape)

In [ ]:
plt.figure()
plt.plot(noise[37, 15, 25, :])
idx = np.argwhere(np.all(noise_reshaped[..., :] == 0, axis=1))
noise_c = np.delete(noise_reshaped, idx, axis=0)
print(noise_c.shape)
print(noise_c.std(axis=1).mean())
print(noise_c.mean(axis=1).mean())
print(np.divide(noise_c.std(axis=1),noise_c.mean(axis=1)).mean())

idx2 = np.argwhere(np.all(volume_reshaped[..., :] == 0, axis=1))
volume_c = np.delete(volume_reshaped, idx2, axis=0)
print(volume_c.shape)
print(volume_c.std(axis=1).mean())
print(volume_c.mean(axis=1).mean())
print(np.divide(volume_c.std(axis=1),volume_c.mean(axis=1)).mean())

**3. Generate signal**

3.1 Specify which voxels in the brain contain signal

In [ ]:
import itertools
x = [i for i in range(noise.shape[0])]
y = [i for i in range(noise.shape[1])]
z = [i for i in range(noise.shape[2])]
coordinates = list(itertools.product(x,y,z))
print(len(coordinates))

In [ ]:
# Create the region of activity where signal will appear
coordinates = np.array(coordinates)  # Where in the brain is the signal
feature_size = 1  # How big, in voxels, is the size of the ROI
signal_volume = fmrisim.generate_signal(dimensions=noise.shape[0:3],
                                        feature_type=['cube'],
                                        feature_coordinates=coordinates,
                                        feature_size=[feature_size],
                                        signal_magnitude=[1],
                                        )

In [ ]:
print(signal_volume)

In [ ]:
signal_volume_c = np.copy(signal_volume)
signal_volume_c[:,:,0] = 1.0
signal_volume_c[:,0,:] = 1.0
signal_volume_c[0,:,:] = 1.0
print(signal_volume_c)

3.2 Characterize signal for voxels

In [ ]:
# Create a pattern for each voxel in our signal ROI
# voxels = feature_size ** 3
voxels = np.count_nonzero(signal_volume_c)
print(voxels)

# Pull the conical voxel activity from a uniform distribution
pattern_A = np.random.rand(voxels).reshape((voxels, 1))
pattern_B = np.random.rand(voxels).reshape((voxels, 1))

In [ ]:
print(pattern_B.shape)

In [ ]:
# Plot pattern of activity for each condition
plt.figure()
plt.subplot(1,2,1)
plt.imshow(pattern_A)
plt.ylabel('Voxels')
plt.tick_params(which='both', left='off', labelleft='off', bottom='off', labelbottom='off')
plt.xlabel('Condition A')

plt.subplot(1,2,2)
plt.imshow(pattern_B)
plt.tick_params(which='both', left='off', labelleft='off', bottom='off', labelbottom='off')
plt.xlabel('Condition B')

3.3 Generate event time course

In [ ]:
# Set up stimulus event time course parameters
event_duration = 2  # How long is each event
isi = 7  # What is the time between each event
burn_in = 1  # How long before the first event

total_time = int(noise.shape[3] * dimsize_c[3]) + burn_in  # How long is the total event time course
events = int((total_time - ((event_duration + isi) * 2))  / ((event_duration + isi) * 2)) * 2  # How many events are there?
onsets_all = np.linspace(burn_in, events * (event_duration + isi), events)  # Space the events out
np.random.shuffle(onsets_all)  # Shuffle their order
onsets_A = onsets_all[:int(events / 2)]  # Assign the first half of shuffled events to condition A
onsets_B = onsets_all[int(events / 2):]  # Assign the second half of shuffled events to condition B
temporal_res = 10.0 # How many timepoints per second of the stim function are to be generated?

In [ ]:
print(total_time)
print(events)
print(onsets_all.shape)

In [ ]:
# Create a time course of events
stimfunc_A = fmrisim.generate_stimfunction(onsets=onsets_A,
                                           event_durations=[event_duration],
                                           total_time=total_time,
                                           temporal_resolution=temporal_res,
                                           )

stimfunc_B = fmrisim.generate_stimfunction(onsets=onsets_B,
                                           event_durations=[event_duration],
                                           total_time=total_time,
                                           temporal_resolution=temporal_res,
                                           )

3.4 Export stimulus time course for analysis

In [ ]:
fmrisim.export_epoch_file(stimfunction=[np.hstack((stimfunc_A, stimfunc_B))],
                          filename=home + '/epoch_file.npy',
                          tr_duration=tr,
                          temporal_resolution=temporal_res,
                          )

fmrisim.export_3_column(stimfunction=stimfunc_A,
                        filename=home + '/Condition_A.txt',
                        temporal_resolution=temporal_res,
                        )

fmrisim.export_3_column(stimfunction=stimfunc_B,
                        filename=home + '/Condition_B.txt',
                        temporal_resolution=temporal_res,
                        )

3.5 Estimate the voxel weight for each event

In [ ]:
# Multiply each pattern by each voxel time course
weights_A = np.matlib.repmat(stimfunc_A, 1, voxels).transpose() * pattern_A
weights_B = np.matlib.repmat(stimfunc_B, 1, voxels).transpose() * pattern_B

# Sum these time courses together
stimfunc_weighted = weights_A + weights_B
stimfunc_weighted = stimfunc_weighted.transpose()

In [ ]:
plt.figure()
plt.plot(stimfunc_weighted[:, 0])
plt.title('Example voxel response time course')
plt.xlabel('Upsampled time course')
plt.ylabel('Response size')

3.6 Convolve each voxel’s time course with the Hemodynamic Response Function

In [ ]:
signal_func = fmrisim.convolve_hrf(stimfunction=stimfunc_weighted,
                                   tr_duration=dimsize_c[3],
                                   temporal_resolution=temporal_res,
                                   scale_function=1,
                                   )

In [ ]:
# Prepare the data to be plotted
response = signal_func[0:100,5] * 2
downsample_A = stimfunc_A[0:int(100*temporal_res * dimsize_c[3]):int(temporal_res * dimsize_c[3]), 0]
downsample_B = stimfunc_B[0:int(100*temporal_res * dimsize_c[3]):int(temporal_res * dimsize_c[3]), 0]

# Display signal
plt.figure()
# plt.title('Example event time course and voxel response')
EventA = plt.plot(downsample_A, 'r', label='Event_A')
EventB = plt.plot(downsample_B, 'g', label='Event_B')
# plt.subplot(2,1,2)
Response = plt.plot(response, 'k', label='Response')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.yticks([],'')
plt.xlabel('nth TR')
plt.savefig('drive/MyDrive/dataset/fig3/event.png',dpi=800)

3.7 Establish signal magnitude


In [ ]:
# Specify the parameters for signal
signal_method = 'CNR_Amp/Noise-SD'
signal_magnitude = [1]

# Where in the brain are there stimulus evoked voxels
signal_idxs = np.where(signal_volume_c == 1)
print(signal_idxs[0].shape)

# Pull out the voxels corresponding to the noise volume
noise_func = noise[signal_idxs[0], signal_idxs[1], signal_idxs[2], :].T

In [ ]:
print(signal_func.shape)
print(noise_func.shape)

In [ ]:
# Compute the signal appropriate scaled
signal_func_scaled = fmrisim.compute_signal_change(signal_func,
                                                  noise_func,
                                                  noise_dict,
                                                  magnitude=signal_magnitude,
                                                  method=signal_method,
                                                  )

In [ ]:
print(signal_func_scaled.shape)
print(noise_func.shape)

idx3 = np.argwhere(np.all(signal_func_scaled[..., :] == 0, axis=0))
signal_c = np.delete(signal_func_scaled, idx, axis=1)
print(signal_c.shape)
print(signal_c.std(axis=0).mean())
print(signal_c.mean(axis=0).mean())
print(np.divide(signal_c.std(axis=0),signal_c.mean(axis=0)).mean())

idx4 = np.argwhere(np.all(noise_func[..., :] == 0, axis=0))
noise_c2 = np.delete(noise_func, idx, axis=1)
print(noise_c2.shape)
print(noise_c2.std(axis=0).mean())
print(noise_c2.mean(axis=0).mean())
print(np.divide(noise_c2.std(axis=0),noise_c2.mean(axis=0)).mean())

In [ ]:
signal_mean = (1*signal_c).mean(axis=0)
noise_std = noise_c2.std(axis=0)
tSNR = np.divide(signal_mean,noise_std)
print('(tSNR)Expected to around 10 (0.03-161.2): ',tSNR.mean())
signal_amp = (1*signal_c).max(axis=0)-(1*signal_c).mean(axis=0)
CNR_amp = np.divide(signal_amp,noise_std)
print('(CNR_amp)Expected to around 37 (8.17-95.73): ',CNR_amp.mean())
signal_std = (1*signal_c).std(axis=0)
CNR_std = np.divide(signal_std,noise_std)
print('(CNR_std)Expected to around 0.029 (0.01-0.79): ',CNR_std.mean())

In [ ]:
signal_mean = (3*signal_c).mean(axis=0)
noise_std = noise_c2.std(axis=0)
tSNR = np.divide(signal_mean,noise_std)
print('(tSNR)Expected to around 10 (0.03-161.2): ',tSNR.mean())
signal_amp = (3*signal_c).max(axis=0)-(3*signal_c).min(axis=0)
CNR_amp = np.divide(signal_amp,noise_std)
print('(CNR_amp)Expected to around 37 (8.17-95.73): ',CNR_amp.mean())
signal_std = (3*signal_c).std(axis=0)
CNR_std = np.divide(signal_std,noise_std)
print('(CNR_std)Expected to around 0.029 (0.01-0.79): ',CNR_std.mean())

 3.8 Multiply the convolved response with the signal voxels

In [ ]:
signal = fmrisim.apply_signal(signal_func_scaled,
                              signal_volume_c,
                              )

In [ ]:
print(signal.shape)

In [ ]:
import random as rd

In [ ]:
print(gm_mask.shape)

In [ ]:
signal_reshaped = np.reshape(signal,[signal.shape[0]*signal.shape[1]*signal.shape[2],signal.shape[3]])
noise_reshaped = np.reshape(noise,[noise.shape[0]*noise.shape[1]*noise.shape[2],noise.shape[3]])
gm_reshaped = np.reshape(gm_mask_c,-1)
ground_truth_list = signal_reshaped[gm_reshaped,:]
print(ground_truth_list.mean())
print(ground_truth_list.mean(axis=1).std())
print(ground_truth_list.std(axis=1).mean())
observation_list = ground_truth_list + noise_reshaped[gm_reshaped,:]
print(ground_truth_list.shape)
print(observation_list.shape)
confounds_mask_reshaped = np.reshape(confounds_mask_c,-1)
noise_list = noise_reshaped[confounds_mask_reshaped,:]
print(noise_list.shape)
print(noise_list.mean())
print(noise_list.mean(axis=1).std())
print(noise_list.std(axis=1).mean())
print(observation_list.mean())
print(observation_list.mean(axis=1).std())
print(observation_list.std(axis=1).mean())

# Preproces of the data

In [ ]:
import numpy as np
import nibabel.processing as nibp
import os
import nibabel as nib
import math
import random
import sklearn.preprocessing
import matplotlib.pyplot as plt
from numpy import random as rd
from sklearn.metrics import mean_squared_error

In [ ]:
class Scaler():
    def __init__(self,inputs):
        self.data = inputs
        self.mean = np.mean(inputs,axis=1)
        self.std = np.std(inputs, axis=1)
        self.vox, self.time = inputs.shape
    def transform(self,inputs):
        self.mean = np.reshape(self.mean,(self.vox,1))
        self.m_large = np.repeat(self.mean,self.time,axis=1)
        self.std = np.reshape(self.std,(self.vox,1))
        self.s_large = np.repeat(self.std,self.time,axis=1)
        return np.divide(inputs-self.m_large,self.s_large)
    def inverse_transform(self,outputs):
        return np.multiply(outputs,self.s_large)+self.m_large

In [ ]:
obs_scale = Scaler(observation_list)
obs_list = obs_scale.transform(observation_list)
noi_scale = Scaler(noise_list)
noi_list = noi_scale.transform(noise_list)
gt_scale = Scaler(ground_truth_list)
gt_list = gt_scale.transform(ground_truth_list)

In [ ]:
x = [i for i in range(noise_list.shape[1])]
plt.figure(figsize = (16,16))
plt.subplot(6,2,1)
plt.plot(x,observation_list[900],'-')
plt.title('signal + noise')
plt.subplot(6,2,2)
plt.plot(x,observation_list[24],'-')
plt.title('signal + noise')
plt.subplot(6,2,3)
plt.plot(x,ground_truth_list[900],'-')
plt.title('ground truth of signal')
plt.subplot(6,2,4)
plt.plot(x,ground_truth_list[24],'-')
plt.title('ground truth of signal')
plt.subplot(6,2,5)
plt.plot(x,noise_list[900],'-')
plt.title('noise, not the noise added to the signal')
plt.subplot(6,2,6)
plt.plot(x,noise_list[24],'-')
plt.title('noise, not the noise added to the signal')

In [ ]:
np.savetxt('drive/MyDrive/dataset/obs_list.csv',obs_list,delimiter=',')
np.savetxt('drive/MyDrive/dataset/gt_list.csv',gt_list,delimiter=',')
np.savetxt('drive/MyDrive/dataset/noi_list.csv',noi_list,delimiter=',')